In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn.functional as F

import sys
sys.path.append('../dgmr-oneshot')
import dgmr

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [9]:
FORECAST = 24
ccs = dgmr.common.ContextConditioningStack(
    input_channels=1,
    conv_type='standard',
    output_channels=160,
)

sampler = dgmr.generators.Sampler(
    forecast_steps=FORECAST,
    latent_channels=96,
    context_channels=160,
    output_channels=1,
)

model = dgmr.generators.Generator(ccs, sampler)
model = model.to(DEVICE)

In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model)} trainable parameters')

The model has 235695682 trainable parameters


In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [12]:
BATCH_SIZE = 16

In [13]:
x = torch.randn(BATCH_SIZE, 4, 1, 128, 128).to(DEVICE)

In [14]:
%time o = model(x)

CPU times: user 300 ms, sys: 128 ms, total: 428 ms
Wall time: 437 ms


In [15]:
o.shape

torch.Size([16, 24, 1, 128, 128])

In [16]:
y = torch.randn(BATCH_SIZE, FORECAST, 1, 128, 128).to(DEVICE)

In [17]:
loss = F.mse_loss(y, o)

In [18]:
loss

tensor(1.1652, device='cuda:0', grad_fn=<MseLossBackward0>)

In [19]:
loss.backward()

In [20]:
optimizer.step()

In [21]:
optimizer.zero_grad()